<center><h1>EMG-based Estimations of Joints Angles and Torquse Using Long Short-Term Memory Network</center></h1>

## Import libraries

In [1]:
import os
# Move back to the main folder, Make sure to run this cell only at the start
os.chdir('..')

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import matplotlib
matplotlib.rcParams['figure.dpi'] = 100
tf.random.set_seed(42)
np.random.seed(42)
plt.figure(dpi=100)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

## define global variables

In [3]:
features_num=88 #Number of features in the model
out_num=9 # Number of outputs
time_steps=50 # time step for training LSTM model

## Create Pipeline

### getting labels function

In [4]:
def get_labels():
    """
    Used to get Output labels as a numpy array. Labels are retrived from Subject 1 dataset
    Labels are ['knee_angle_r', 'ankle_angle_r', 'knee_angle_l', 'ankle_angle_l',
            'knee_angle_r_moment', 'knee_angle_l_moment',
            'ankle_angle_r_moment', 'ankle_angle_l_moment']
    Returns
    -------
    list contains all output labels

    """

    output_label =["Right Knee Angle", "Left Knee Angle", "Right Hip Angle",
                "Left Hip Angle", "Right Knee Torque", "Left Knee Torque", "Right Hip Torque", "Left Hip Torque", "Shank Gyro Y IMU"]
    return output_label

### dataset pipeline

In [5]:
def get_dataset(subject_no):
    """
    This function is used to retrive the dataset for selected subject. Dataset
    will and last 8 columns contains the output. All columns are labeled.

    Parameters
    ----------
    subject_no : string
    Subject number in format XX.

    Returns
    -------
    pandas dataset
    Subject number "subject_no" dataset without time column
    
    """
    subject_no = "01"
    dataset = pd.read_csv('Subject' + subject_no + '_dataset_20200207_50ms.csv',header=0)
    return dataset.drop(columns="Time").dropna(axis=0)

In [6]:
def get_scaler(ndarray, features_range=(-1,1)):
    """
    This function is used for creating scalers for scalling the features using
    sklearn.preprocessing.MinMaxScaler function.

    Parameters
    ----------
    ndarray : numpy arrayor pandas dataset
    DESCRIPTION.
    scale_range : Tuple, optional
    Scaling range (Min,Max). The default is (-1,1).

    Returns
    -------
    scaler : MinMaxscaler
    """
    scaler = MinMaxScaler(feature_range=features_range)
    scaler.fit(ndarray)
    return scaler

In [7]:
def scale_data(dataset, features_range=(-1,1), outputs_range=None):
    """
    Scale input/output using MinMaxScaler. inputs are scaled in range (-1,1)
    and outputs scaled in general model case only using range of (0,1)

    Parameters
    ----------
    dataset : numpy array or pandas
      Dataset to be scaled.
    features_range : tuple
      The scaler range for input features. Default (-1,-1)
    outputs_range : tuple or None
      Scale all outputs in range the desired range. If None, no scaling will
      be performed. Default value is None.

    Returns
    -------
    scaled_dataset : numpy array
      a numpy array contains the scaled dataset.
    output_scaler : MinMaxscaler
      Output values scaler. Optional if Outpus_range is not None

    """
    dataset_values = dataset.values.astype("float32")
    input_scaler = get_scaler(dataset_values[:,:-out_num]) #get scaler for input features
    input_dataset = input_scaler.transform(dataset_values[:,:-out_num]) #scale input features
    if outputs_range==None:
        output_dataset = dataset_values[:,-out_num:]
        scaled_dataset = np.concatenate((dataset_values[:,:-out_num],output_dataset), axis=1)
        return scaled_dataset
    else:
        output_scaler = get_scaler(dataset_values[:,-out_num:], scale_range=outputs_range)
        output_dataset = output_scaler.transform(dataset_values[:,-out_num:])
        scaled_dataset = np.concatenate((dataset_values[:,:-out_num],output_dataset), axis=1)
        return scaled_dataset, output_scaler

In [8]:
def create_inout_sequences(data_set):
    """
    Create the input and output pairs sequances for the model

      Parameters
    ----------
    input_data : numpy array
        Subject dataset pair (inputs & Ooutputs).
    features : int
        Number of features in our model. default is 112
    out_num : int
        Number of outputs for the model to predict. outputs are arranged same way the get_label() function output.
        default value is 8
    time_steps : int
        Time step for training LSTM model. default value is 50

    Returns
    -------
    features : numpy array
        Numpy array with the shape (len(data_set)-time_step ,time_steps, features) contains the features.
    outputs : numpy array
        Numpy array with the shape (len(data_set)-time_step ,time_steps, out_num) contains all ouputs corrsponding to the features
    """
    features = []
    outputs =  []
    L = len(data_set)
    for i in range(L-time_steps):
        inputs = data_set[i:i+time_steps,:features_num]
        labels = data_set[i:i+time_steps,-out_num:]
        features.append(inputs)
        outputs.append(labels)
    features = np.array(features)
    outputs = np.array(outputs)
    return features,outputs

## Create a function that will process all the pipeline

In [9]:
def in_out_process(subject, mod="training_set", test_val_size=0.4, features_range=(-1,1), outputs_range=None):
    """
    This function will process the dataset and give us the training, validation and test sets without shuffeling 

    Parameters
    ----------
    subject: an "XX" string format.
    mod: a string that tells the function about the goal of the recieved data set, is it for training or testing. Either "training_set" or "test_set".
    test_val_size: float between [0,1), show the percentage of the validation and test sets from the total data set.
      data will be as follow:
      from 0 to (1-test_val_size)% for training
      from (1-test_val_size)% to (1-test_val_size/2) for validation
      from (1-test_val_size/2)% till the end of the data set for testing
    features_range : tuple
      The scaler range for input features. Default (-1,-1)
    outputs_range : tuple or None
      Scale all outputs in range the desired range. If None, no scaling will
      be performed. Default value is None.

    Returns:
    X_train, X_val, y_train, y_val arrays as float32 tensors if mod is set for "training_set"
    X_test, y_test if mod is set for "test_set"
    """
    # in case of training
    if mod=="training_set":
        # Load the dataset
        dataset = get_dataset(subject)
        # Scale the dataset
        scaled_dataset = scale_data(dataset, features_range=features_range
                                   , outputs_range=None)
        # get features and outputs
        features, outputs = create_inout_sequences(scaled_dataset)
        # get training set
        X_train, X_test_val, y_train, y_test_val = train_test_split(features, outputs,
                                                          test_size = test_val_size,
                                                          random_state=None, shuffle=False)
        # get validation
        X_val, _, y_val, _ = train_test_split(features, outputs,
                                              test_size = test_val_size/2,
                                              random_state=None, shuffle=False)

        print("Dataset_size: %2d, train_size: %2d, test_size: %2d" %(len(features),
                                                                    len(X_train),
                                                                    len(X_val)))

        return tf.constant(X_train,tf.float32), tf.constant(X_val,tf.float32), tf.constant(y_train,tf.float32), tf.constant(y_val,tf.float32)
    # in case of testing
    elif mod =="test_set":
        # Load the dataset
        dataset = get_dataset(subject)
        # Scale the dataset
        scaled_dataset = scale_data(dataset)
        # get test set starting point
        test_len = int((1-test_val_size/2)*len(scaled_dataset))
        # get test set
        X_test = scaled_dataset[test_len: ,:-out_num]
        y_test = scaled_dataset[test_len:,-out_num:]
        return X_test, y_test

## Model creation and Evaluation

### define a function to create the model

In [10]:
def create_model():
    keras.backend.clear_session() # Make sure we do not have any model in the notebook
    model = keras.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(4, return_sequences=True, input_shape=(time_steps, features_num),
                              dropout=0.3), name="input_layer"))
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(4, return_sequences=True,
                              dropout=0.3), name="2nd_layer"))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(out_num), name="output_layer"))
    return model

### Model Evaluation function

In [11]:
def col_arranger(df):
    """
    Arrange columns to match the paper
    """
    DF = df[['Right Knee Angle', 'Left Knee Angle', 'Right Hip Angle',
                'Left Hip Angle', 'Right Knee Torque', 'Left Knee Torque', 'Right Hip Torque', 'Left Hip Torque', 'Shank Gyro Y IMU']]
    return DF

def rmse_cal(Y_pred,Y):
    """
    This function will calculate RMSE values
    """
    size = Y_pred.shape[0]
    rmse_value = np.sqrt(np.sum((Y_pred - Y)**2)/size)
    return rmse_value

def R2_cal(Y_pred,Y):
    """
    This function will calculate R2 scores
    """
    size = Y_pred.shape[0]
    R2_value = 1 - np.sum((Y_pred - Y)**2)/np.sum(Y**2)
    return R2_value
        
#Get the predicted values from the test dataset
def get_prediction(inputs, model):
    """
    get the predictions from inputs
    inputs came in shape (length of the test set, features_num)
    """
    m = len(inputs)
    for i in range(0,m-50,50):
        X = inputs[i:i+50,:]
        len(X)
        print(len(X))
        X = np.reshape(X,(1,50,88))
        if i == 0:
            Y_preds = model.predict(X) 
        else:
            y_pred = model.predict(X)
            Y_preds = np.concatenate((Y_preds, y_pred))
    return np.reshape(Y_preds,(-1,out_num))

def evaluation(Y, Y_pred, Folder, labels=get_labels()):   
    """
    Evaluate the model and plot the results
    """ 
    time = [i*0.05 for i in range(len(Y_pred))]
    frame_size=len(time)
    r2_score = []
    RMSE_score = []
    for col in range(out_num):
        rmse = np.round(rmse_cal(Y[:frame_size, col], Y_pred[:frame_size, col]),2)
        R2 = np.round(R2_cal(Y[:frame_size, col],Y_pred[:frame_size, col]),4)
        RMSE_score.append(rmse)
        r2_score.append(R2)
        if col == 0 or col == 1 or col == 2 or col == 3:
            y_label = 'Degree'
            unit = '\xB0'
        else:
            y_label = 'Nm'
            unit = ' Nm'
        print('R2_score' + f'({labels[col]}) =', R2)
        print('RMSE' + f'({labels[col]}) = {rmse}' + unit)
        plt.plot(time[:frame_size], Y[:frame_size, col], 'r')
        plt.plot(time[:frame_size], Y_pred[:frame_size, col], 'b--')
        plt.xlabel('Time [s]', fontsize=14)
        plt.xlim((0,30))
        plt.ylabel(y_label, fontsize=14)
        plt.xticks(fontsize=11)
        plt.yticks(fontsize=11)
        plt.title(labels[col], fontsize="xx-large")
        plt.savefig(Folder + '/' + f'{labels[col]}.pdf')
        plt.show()

    return r2_score, RMSE_score

## Train and Evaluate the model

In [12]:
def train_eval(subject=1, lr=0.001, test_val_size=0.4,
               load_best=False, eval_only=False, features_range=(-1, 1)):
    """
    This function will train and evaluated the model
    load_best: to load trainable saved model
    eval_only: to skip training process and evaluate the model only, if it set to True, load_best will be True
    """
 
    global model_path # Create model folder and path and make them global
    #Folder = "S" + subject
    #if not os.path.exists(Folder): #if folder doesn't exist, create one
        #os.makedirs(Folder)
    model_path = r"C:/Users/ibra5/Desktop/LSTM/Model Output_05/model.hdf5" # Model path and name

    output_label = get_labels() #import the labels
    if not eval_only: #load dataset for training the model
        X_train, X_val, y_train, y_val = in_out_process(subject, mod="training_set",
                                                        test_val_size=test_val_size, 
                                                        features_range=features_range)

    model = create_model() # Create the model
    try:
        if load_best or eval_only: #load best model if we will evaluate the model only our we want to start training from pre-trained model
            if os.path.exists(model_path): # make sure model exist
                model.load_weights(model_path)
            else:
                pass

          # Train the model
        if not eval_only:
            model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="MSE") # Create model Compiler
            # Define Check points for saving best model while training
            model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( 
            filepath = model_path,
            save_weights_only=True,
            monitor='val_loss',
            save_best_only=True)

            training = model.fit(x=X_train, y=y_train, batch_size=64, epochs=6500,
                              validation_data=(X_val,y_val), 
                              callbacks=[model_checkpoint_callback])
      
            # Plot learning curve after the end of training
            print("\ntrain end\n\n Plotting Learning Curve")
            plt.plot(training.history["loss"][100:])
            plt.plot(training.history["val_loss"][100:])
            plt.title("Model loss")
            plt.ylabel("loss")
            plt.xlabel("epoch")
            plt.legend(["train", "val"], loc="upper left")
            plt.show()

    except KeyboardInterrupt:
        print("\nTrain manually stopped\n\n")

    #Evaluate the model
    # Get test set
    X_test, y_test = in_out_process(subject, mod="test_set", test_val_size=test_val_size, features_range=(-1,1))
    # Get predictions
    y_test_preds = get_prediction(X_test, model)
    # Get R2 and RMSE results and plot measurements against predictions
    r2_score, RMSE_score = evaluation(y_test, y_test_preds, Folder=r"C:/Users/ibra5/Desktop/LSTM/Model Output_05", labels=output_label)

    # Return results as a pandas dataframe
    r2_score = pd.DataFrame(data=np.array(r2_score,ndmin=2), columns=output_label)
    r2_score.index = {"S" + subject}

    RMSE_score = pd.DataFrame(data=np.array(RMSE_score, ndmin=2), columns=output_label)
    RMSE_score.index = {"S" + subject}
    # arrange columns and get the output
    return col_arranger(r2_score), col_arranger(RMSE_score)

## Train model



In [ ]:
S1_r2, S1_rmse = train_eval(subject = "01", load_best=False, lr=0.001, eval_only=False)

Dataset_size: 2809, train_size: 1685, test_size: 2247
Epoch 1/6500
27/27 [==============================] - 11s 138ms/step - loss: 548.2346 - val_loss: 553.1248
Epoch 2/6500
27/27 [==============================] - 1s 48ms/step - loss: 541.0104 - val_loss: 543.4117
Epoch 3/6500
27/27 [==============================] - 2s 62ms/step - loss: 531.5760 - val_loss: 532.4789
Epoch 4/6500
27/27 [==============================] - 3s 100ms/step - loss: 520.2256 - val_loss: 521.0638
Epoch 5/6500
27/27 [==============================] - 1s 47ms/step - loss: 508.5036 - val_loss: 509.9325
Epoch 6/6500
27/27 [==============================] - 2s 59ms/step - loss: 497.3471 - val_loss: 499.2501
Epoch 7/6500
27/27 [==============================] - 3s 100ms/step - loss: 486.9511 - val_loss: 489.3055
Epoch 8/6500
27/27 [==============================] - 1s 49ms/step - loss: 477.3719 - val_loss: 480.1414
Epoch 9/6500
27/27 [==============================] - 2s 57ms/step - loss: 468.4174 - val_loss: 471.05

27/27 [==============================] - 2s 57ms/step - loss: 274.7387 - val_loss: 281.9506
Epoch 78/6500
27/27 [==============================] - 3s 100ms/step - loss: 274.0560 - val_loss: 281.2761
Epoch 79/6500
27/27 [==============================] - 3s 96ms/step - loss: 273.4144 - val_loss: 280.6283
Epoch 80/6500
27/27 [==============================] - 1s 56ms/step - loss: 272.7827 - val_loss: 280.0065
Epoch 81/6500
27/27 [==============================] - 3s 100ms/step - loss: 272.1909 - val_loss: 279.4105
Epoch 82/6500
27/27 [==============================] - 3s 99ms/step - loss: 271.6142 - val_loss: 278.8353
Epoch 83/6500
27/27 [==============================] - 3s 100ms/step - loss: 271.0641 - val_loss: 278.2866
Epoch 84/6500
27/27 [==============================] - 2s 57ms/step - loss: 270.5405 - val_loss: 277.7546
Epoch 85/6500
27/27 [==============================] - 3s 96ms/step - loss: 270.0265 - val_loss: 277.2472
Epoch 86/6500
27/27 [==============================] - 2s

27/27 [==============================] - 3s 99ms/step - loss: 260.1695 - val_loss: 267.0469
Epoch 154/6500
27/27 [==============================] - 3s 99ms/step - loss: 260.1601 - val_loss: 267.0351
Epoch 155/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.1523 - val_loss: 267.0233
Epoch 156/6500
27/27 [==============================] - 2s 58ms/step - loss: 260.1416 - val_loss: 267.0120
Epoch 157/6500
27/27 [==============================] - 3s 103ms/step - loss: 260.1377 - val_loss: 267.0014
Epoch 158/6500
27/27 [==============================] - 3s 95ms/step - loss: 260.1304 - val_loss: 266.9917
Epoch 159/6500
27/27 [==============================] - 2s 56ms/step - loss: 260.1248 - val_loss: 266.9831
Epoch 160/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.1221 - val_loss: 266.9741
Epoch 161/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.1136 - val_loss: 266.9670
Epoch 162/6500
27/27 [===========================

27/27 [==============================] - 3s 98ms/step - loss: 260.0569 - val_loss: 266.8689
Epoch 230/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0579 - val_loss: 266.8689
Epoch 231/6500
27/27 [==============================] - 2s 88ms/step - loss: 260.0583 - val_loss: 266.8691
Epoch 232/6500
27/27 [==============================] - 2s 59ms/step - loss: 260.0570 - val_loss: 266.8686
Epoch 233/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0592 - val_loss: 266.8701
Epoch 234/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0571 - val_loss: 266.8664
Epoch 235/6500
27/27 [==============================] - 3s 99ms/step - loss: 260.0587 - val_loss: 266.8677
Epoch 236/6500
27/27 [==============================] - 1s 55ms/step - loss: 260.0579 - val_loss: 266.8698
Epoch 237/6500
27/27 [==============================] - 2s 94ms/step - loss: 260.0563 - val_loss: 266.8672
Epoch 238/6500
27/27 [==============================

27/27 [==============================] - 3s 97ms/step - loss: 260.0552 - val_loss: 266.8692
Epoch 306/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0518 - val_loss: 266.8692
Epoch 307/6500
27/27 [==============================] - 2s 76ms/step - loss: 260.0521 - val_loss: 266.8660
Epoch 308/6500
27/27 [==============================] - 2s 71ms/step - loss: 260.0517 - val_loss: 266.8664
Epoch 309/6500
27/27 [==============================] - 3s 101ms/step - loss: 260.0533 - val_loss: 266.8670
Epoch 310/6500
27/27 [==============================] - 3s 102ms/step - loss: 260.0522 - val_loss: 266.8657
Epoch 311/6500
27/27 [==============================] - 1s 49ms/step - loss: 260.0517 - val_loss: 266.8673
Epoch 312/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0509 - val_loss: 266.8714
Epoch 313/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0526 - val_loss: 266.8690
Epoch 314/6500
27/27 [============================

...

27/27 [==============================] - 3s 100ms/step - loss: 260.0428 - val_loss: 266.8612
Epoch 838/6500
27/27 [==============================] - 2s 77ms/step - loss: 260.0440 - val_loss: 266.8663
Epoch 839/6500
27/27 [==============================] - 2s 72ms/step - loss: 260.0435 - val_loss: 266.8640
Epoch 840/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0437 - val_loss: 266.8612
Epoch 841/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0428 - val_loss: 266.8620
Epoch 842/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0432 - val_loss: 266.8641
Epoch 843/6500
27/27 [==============================] - 1s 51ms/step - loss: 260.0429 - val_loss: 266.8615
Epoch 844/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.0436 - val_loss: 266.8602
Epoch 845/6500
27/27 [==============================] - 3s 101ms/step - loss: 260.0435 - val_loss: 266.8602
Epoch 846/6500
27/27 [===========================

27/27 [==============================] - 3s 98ms/step - loss: 260.0425 - val_loss: 266.8629
Epoch 914/6500
27/27 [==============================] - 2s 62ms/step - loss: 260.0424 - val_loss: 266.8587
Epoch 915/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0422 - val_loss: 266.8629
Epoch 916/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0426 - val_loss: 266.8642
Epoch 917/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.0440 - val_loss: 266.8595
Epoch 918/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0430 - val_loss: 266.8646
Epoch 919/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0446 - val_loss: 266.8693
Epoch 920/6500
27/27 [==============================] - 2s 87ms/step - loss: 260.0430 - val_loss: 266.8577
Epoch 921/6500
27/27 [==============================] - 2s 61ms/step - loss: 260.0423 - val_loss: 266.8594
Epoch 922/6500
27/27 [=============================

27/27 [==============================] - 2s 56ms/step - loss: 260.0419 - val_loss: 266.8635
Epoch 990/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0418 - val_loss: 266.8587
Epoch 991/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0423 - val_loss: 266.8609
Epoch 992/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0429 - val_loss: 266.8631
Epoch 993/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0425 - val_loss: 266.8590
Epoch 994/6500
27/27 [==============================] - 2s 56ms/step - loss: 260.0439 - val_loss: 266.8622
Epoch 995/6500
27/27 [==============================] - 3s 102ms/step - loss: 260.0424 - val_loss: 266.8589
Epoch 996/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0427 - val_loss: 266.8572
Epoch 997/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0409 - val_loss: 266.8626
Epoch 998/6500
27/27 [=============================

Epoch 1065/6500
27/27 [==============================] - 1s 49ms/step - loss: 260.0419 - val_loss: 266.8630
Epoch 1066/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.0412 - val_loss: 266.8602
Epoch 1067/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0414 - val_loss: 266.8638
Epoch 1068/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0410 - val_loss: 266.8569
Epoch 1069/6500
27/27 [==============================] - 3s 98ms/step - loss: 260.0415 - val_loss: 266.8619
Epoch 1070/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0419 - val_loss: 266.8579
Epoch 1071/6500
27/27 [==============================] - 1s 51ms/step - loss: 260.0412 - val_loss: 266.8612
Epoch 1072/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.0414 - val_loss: 266.8615
Epoch 1073/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0422 - val_loss: 266.8622
Epoch 1074/6500
27/27 [===

27/27 [==============================] - 3s 98ms/step - loss: 260.0408 - val_loss: 266.8600
Epoch 1141/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0415 - val_loss: 266.8669
Epoch 1142/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.0424 - val_loss: 266.8561
Epoch 1143/6500
27/27 [==============================] - 3s 100ms/step - loss: 260.0402 - val_loss: 266.8594
Epoch 1144/6500
27/27 [==============================] - 1s 51ms/step - loss: 260.0408 - val_loss: 266.8582
Epoch 1145/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0417 - val_loss: 266.8553
Epoch 1146/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0414 - val_loss: 266.8572
Epoch 1147/6500
27/27 [==============================] - 3s 97ms/step - loss: 260.0407 - val_loss: 266.8595
Epoch 1148/6500
27/27 [==============================] - 3s 95ms/step - loss: 260.0423 - val_loss: 266.8604
Epoch 1149/6500
27/27 [===================

In [ ]:
S1_r2

In [ ]:
S1_rmse